In [4]:
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cpu


In [1]:
class FeedForward(nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.W1 = nn.Linear(512, 2048)
        self.W2 = nn.Linear(2048, 512)
        self.ffStack = nn.Sequential(
            self.W1(),
            nn.ReLU(),
            self.W2()
        )
    def forward(self, x):
        return self.ffStack(x)

class SelfAttention(nn.Module):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def forward(self, K, V, Q):
        d_k = Q.shape()[0]
        return F.softmax((Q@K.T) * (1/torch.sqrt(d_k)), axis=1) @ V











NameError: name 'nn' is not defined